In [1]:
import sys 
sys.path.append("/scratch/m/murray/dtolgay/")
from tools import functions_readfiles


import numpy as np 
import pandas as pd 
from tools import readsnap, readsnap_FIREBox, readsnap_tripleLatte, constants  # tools directory is in the appended system directory
from tools.filter_rotate_galaxy import filter_rotate_galaxy

import matplotlib.pyplot as plt 
import matplotlib

import h5py
from time import localtime 


# Main

In [6]:
redshift = "1.0"

if redshift == "0.0":
    snapshot_number = "1200"     # z = 0.0
    ahf_file_name = "FB15N1024.z0.000.AHF_halos"        
elif redshift == "0.5":
    print(f"Exiting... Currently there are no z=0.5 galaxies... {redshift}")
    sys.exit(2)                
elif redshift == "1.0":
    snapshot_number = "554"     # z = 1.0
    ahf_file_name = "FB15N1024.z1.000.AHF_halos"        
elif redshift == "2.0":
    snapshot_number = "344"     # z = 2.0
    ahf_file_name = "FB15N1024.z2.000.AHF_halos"        
elif redshift == "3.0":
    snapshot_number = "240"     # z = 3.0
    ahf_file_name = "FB15N1024.z3.000.AHF_halos"        
else:
    print(f"Exiting... Redshift is wrong. The given galaxy type is {redshift}")
    sys.exit(2)      

In [8]:
%%time

snap_dir_file_path = "/scratch/m/murray/dtolgay/firebox/FB15N1024"

gas_particles  = readsnap.readsnap(snap_dir_file_path, snapshot_number, 0, cosmological=1)
star_particles = readsnap.readsnap(snap_dir_file_path, snapshot_number, 4, cosmological=1)

print(gas_particles.keys())




npart_file:  [12145723 13575286        0        0  1534311        0]
npart_total: [ 964819983 1073741824          0          0  110115768          0]
npart_file:  [12145723 13575286        0        0  1534311        0]
npart_total: [ 964819983 1073741824          0          0  110115768          0]
dict_keys(['k', 'p', 'v', 'm', 'id', 'u', 'rho', 'h', 'ne', 'nh', 'sfr', 'z'])
CPU times: user 11min 56s, sys: 6min 12s, total: 18min 8s
Wall time: 19min 6s


In [9]:
header_info = readsnap.readsnap(snap_dir_file_path, snapshot_number, 0, header_only=1)

hubble      = float(header_info['hubble'])
redshift_from_header    = float(header_info['redshift'])   
time        = float(header_info['time'])   

header_info


npart_file:  [12145723 13575286        0        0  1534311        0]
npart_total: [ 964819983 1073741824          0          0  110115768          0]


{'k': 0,
 'time': 0.49999999994757255,
 'boxsize': 15000.0,
 'hubble': 0.6774,
 'npart': array([12145723, 13575286,        0,        0,  1534311,        0],
       dtype=int32),
 'npartTotal': array([ 964819983, 1073741824,          0,          0,  110115768,
                 0], dtype=uint32),
 'redshift': 1.00000000020971}

In [10]:
%%time

gas_particles_df = pd.DataFrame({
    'x': gas_particles['p'][:,0],  # kpc 
    'y': gas_particles['p'][:,1],  # kpc 
    'z': gas_particles['p'][:,2],  # kpc 
    'vx': gas_particles['v'][:,0],
    'vy': gas_particles['v'][:,1],
    'vz': gas_particles['v'][:,2],
    'mass': gas_particles['m'],
    'density': gas_particles['rho'],
    'smoothing_length': gas_particles['h'],
    'star_formation_rate': gas_particles['sfr'],
    'internal_energy': gas_particles['u'] * 1e6,  # Converted to [m^2 s^-2]
    'neutral_hydrogen_fraction': gas_particles['nh'],
    'electron_abundance': gas_particles['ne'],
    'metallicity': gas_particles['z'][:,0],
    'He_mass_fraction': gas_particles['z'][:,1]
    # You can add other fractions as needed
})


# Create dataframe for star particles
star_particles_df = pd.DataFrame({
    'x': star_particles['p'][:,0],
    'y': star_particles['p'][:,1],
    'z': star_particles['p'][:,2],
    'vx': star_particles['v'][:,0],
    'vy': star_particles['v'][:,1],
    'vz': star_particles['v'][:,2],
    'metallicity': star_particles['z'][:,0],
    'scale_factor': star_particles['age'],
    'mass': star_particles['m']
})

CPU times: user 2min 33s, sys: 2min 59s, total: 5min 33s
Wall time: 5min 34s


In [ ]:
#################################### Plot

print("Plotting galaxy")
# Create a figure with two subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))  # Adjust figsize as needed

# First subplot: x-y axis plot
axs[0].hist2d(
    x=gas_particles_df["x"], 
    y=gas_particles_df["y"], 
    bins=500,
    norm=matplotlib.colors.LogNorm(),
)
axs[0].set_title('x-y axis plot')
axs[0].set_xlabel('x')
axs[0].set_ylabel('y')

# Second subplot: z-y axis plot
axs[1].hist2d(
    x=gas_particles_df["y"],  # Use the 'y' column for the x-axis
    y=gas_particles_df["z"],  # Keep the 'z' column for the y-axis
    bins=500,
    norm=matplotlib.colors.LogNorm(),
#     range=[[-R_max, R_max], [-R_max, R_max]]
)
axs[1].set_title('z-y axis plot')
axs[1].set_xlabel('y')
axs[1].set_ylabel('z')

# Show the figure
plt.tight_layout()


# Read the AHF files

In [11]:
fdir = f"/scratch/m/murray/dtolgay/firebox/FB15N1024/analysis/AHF/halo_new/{snapshot_number}"
# file_name = "FB15N1024.z0.000.AHF_halos"
file_name = "FB15N1024.z1.000.AHF_halos"

column_names = [
    "ID", "hostHalo", "numSubStruct", "Mvir", "npart", "Xc", "Yc", "Zc", "VXc", "VYc", "VZc",
    "Rvir", "Rmax", "r2", "mbp_offset", "com_offset", "Vmax", "v_esc", "sigV", "lambda", "lambdaE",
    "Lx", "Ly", "Lz", "b", "c", "Eax", "Eay", "Eaz", "Ebx", "Eby", "Ebz", "Ecx", "Ecy", "Ecz",
    "ovdens", "nbins", "fMhires", "Ekin", "Epot", "SurfP", "Phi0", "cNFW", "n_gas", "M_gas",
    "lambda_gas", "lambdaE_gas", "Lx_gas", "Ly_gas", "Lz_gas", "b_gas", "c_gas", "Eax_gas",
    "Eay_gas", "Eaz_gas", "Ebx_gas", "Eby_gas", "Ebz_gas", "Ecx_gas", "Ecy_gas", "Ecz_gas",
    "Ekin_gas", "Epot_gas", "n_star", "M_star", "lambda_star", "lambdaE_star", "Lx_star",
    "Ly_star", "Lz_star", "b_star", "c_star", "Eax_star", "Eay_star", "Eaz_star", "Ebx_star",
    "Eby_star", "Ebz_star", "Ecx_star", "Ecy_star", "Ecz_star", "Ekin_star", "Epot_star"
]

halos = pd.DataFrame(
    np.loadtxt(f"{fdir}/{file_name}"),
    columns=column_names
)

# Reorder the dataframe according to the Mvir 
halos = halos.sort_values(by="Mvir", ascending=False).reset_index()

In [12]:
np.log10(halos["Mvir"])

halos.iloc[100:200]

,index,ID,hostHalo,numSubStruct,Mvir,npart,Xc,Yc,Zc,VXc,...,Eay_star,Eaz_star,Ebx_star,Eby_star,Ebz_star,Ecx_star,Ecy_star,Ecz_star,Ekin_star,Epot_star
100,101,101.0,-1.0,21.0,1.313230e+11,980142.0,6567.478180,12005.710602,3249.721527,-30.22,...,-0.325971,0.944666,0.941035,0.329410,0.077086,-0.336311,0.886133,0.318847,3.326640e+13,-1.191960e+14
101,96,96.0,-1.0,15.0,1.304850e+11,1025447.0,551.433563,14453.487396,5924.205780,-64.21,...,0.980224,-0.190486,0.995083,0.068373,0.071658,-0.083265,0.185706,0.979071,1.817380e+13,-8.088790e+13
102,120,120.0,-1.0,23.0,1.300400e+11,809861.0,1618.595123,14218.082428,729.274750,42.36,...,0.825070,0.391891,0.047151,-0.409490,0.911096,0.912192,0.389331,0.127776,1.542420e+13,-7.374320e+13
103,110,110.0,-1.0,21.0,1.291700e+11,862849.0,4513.149261,14441.471100,11269.397736,35.93,...,0.320705,0.908754,-0.417809,0.888294,-0.190707,0.868402,0.328758,-0.371210,4.087020e+13,-1.859770e+14
104,99,99.0,-1.0,17.0,1.287650e+11,996577.0,5348.682404,12832.202911,13636.722565,41.00,...,0.844724,0.068178,0.777397,0.517402,-0.357701,0.337434,0.136881,0.931344,5.088690e+13,-2.086060e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,195.0,-1.0,14.0,7.436240e+10,528286.0,9984.626770,7401.695251,2724.952698,-24.77,...,-0.261432,0.790986,0.833005,0.185389,-0.521280,-0.010361,0.947251,0.320325,6.459290e+12,-2.292910e+13
196,193,193.0,-1.0,12.0,7.412300e+10,532944.0,2350.730896,14306.373596,12158.088684,-5.71,...,0.845733,-0.145623,-0.526943,0.444571,0.724354,0.677350,-0.295113,0.673874,3.359220e+12,-1.608850e+13
197,211,211.0,-1.0,5.0,7.381760e+10,485651.0,14751.520157,10154.943466,9682.245255,52.95,...,0.811445,0.582954,0.998066,-0.060698,0.013453,-0.046300,-0.581269,0.812393,1.569700e+13,-6.807290e+13
198,174,174.0,-1.0,12.0,7.370640e+10,567456.0,10504.417419,14670.753479,10161.323547,95.73,...,-0.230737,0.924814,0.425834,0.835335,0.347680,-0.852752,0.498975,-0.154396,4.749620e+12,-2.332690e+13


## Find the gas particles in each halo

In [13]:
%%time

import functions_AHF

import importlib
importlib.reload(functions_AHF)

# Initiate an empty array to store the halos
halos_of_interest = []

start_halo = 100
# Find the first 100 halos 
for row, halo in halos.iloc[start_halo:start_halo+100].iterrows():
        
    print(f" ----------------------- {row} -----------------------")
    now = localtime()
    print(f"{now.tm_year}/{now.tm_mon}/{now.tm_mday} at {now.tm_hour}:{now.tm_min}")
    
    # if hostHalo is zero, that means it is the top level halo
    if (int(halo['hostHalo']) == -1): 
        
        x_center = halo['Xc'] # kpc/h
        y_center = halo['Yc'] # kpc/h
        z_center = halo['Zc'] # kpc/h
        rvir = halo['Rvir'] # kpc/h
        mvir = halo["Mvir"]


        # The positions and mass of the Halo is in comoving units. To make everything on physical units the below code is written
        x_center = x_center * time/hubble     # [kpc]
        y_center = y_center * time/hubble     # [kpc]
        z_center = z_center * time/hubble     # [kpc]  
        rvir = rvir * time/hubble     # [kpc]      
        mvir = mvir * 1/hubble # [Msolar]

        halos_of_interest.append(
            {
                "x": x_center,
                "y": y_center,
                "z": z_center,
                "rvir": rvir,
                "mvir": mvir,
            }
        )

        # TODO: Change it to rvir 
        rmax = 50 # kpc 
        filtered_gas_particles = functions_AHF.filter_particles(particles_df = gas_particles_df, x_halo = x_center, y_halo = y_center, z_halo = z_center, rmax = rmax)
        filtered_star_particles = functions_AHF.filter_particles(particles_df = star_particles_df, x_halo = x_center, y_halo = y_center, z_halo = z_center, rmax = rmax)

        # Change the origin of the particles 
        filtered_gas_particles = functions_AHF.change_origin(
            particles = filtered_gas_particles.copy(),
            x_halo_center = x_center, 
            y_halo_center = y_center, 
            z_halo_center = z_center,
        )

        filtered_star_particles = functions_AHF.change_origin(
            particles = filtered_star_particles.copy(), 
            x_halo_center = x_center, 
            y_halo_center = y_center,
            z_halo_center = z_center, 
        )

        # Reconstruct the dictionary to write the file 
        reconstructed_gas_particles = {
            'p': filtered_gas_particles[['x', 'y', 'z']].to_numpy(),  # shape (N, 3)
            'v': filtered_gas_particles[['vx', 'vy', 'vz']].to_numpy(),  # shape (N, 3)
            'm': filtered_gas_particles['mass'].to_numpy(),  # shape (N,)
            'rho': filtered_gas_particles['density'].to_numpy(),
            'h': filtered_gas_particles['smoothing_length'].to_numpy(),
            'sfr': filtered_gas_particles['star_formation_rate'].to_numpy(),
            'u': (filtered_gas_particles['internal_energy'] / 1e6).to_numpy(),  # reverse conversion
            'nh': filtered_gas_particles['neutral_hydrogen_fraction'].to_numpy(),
            'ne': filtered_gas_particles['electron_abundance'].to_numpy(),
            'z': filtered_gas_particles[['metallicity', 'He_mass_fraction']].to_numpy()  # shape (N, 2)
        }

        reconstructed_star_particles = {
            'p': filtered_star_particles[['x', 'y', 'z']].to_numpy(),         # shape (N, 3)
            'v': filtered_star_particles[['vx', 'vy', 'vz']].to_numpy(),      # shape (N, 3)
            'z': filtered_star_particles[['metallicity']].to_numpy(),         # shape (N, 1) — or (N,) depending on original
            'age': filtered_star_particles['scale_factor'].to_numpy(),        # shape (N,)
            'm': filtered_star_particles['mass'].to_numpy()                   # shape (N,)
        }    

        # Save galaxies into a hdf5 file
        file_name = f"gal_{row}.hdf5"
        with h5py.File(f'z{redshift}/{file_name}', 'w') as f: 
            gas_group = f.create_group('gas')
            for key, value in reconstructed_gas_particles.items():
                gas_group.create_dataset(key, data=value)

            stars_group = f.create_group('star')
            for key, value in reconstructed_star_particles.items():
                stars_group.create_dataset(key, data=value)

        print(f"{file_name} written!")
            
            
    else: 
        print(halo)
            
##     Delete below if statement. It is done for debugging.
#     if row >= 0:
#         break 

    
halos_of_interest = pd.DataFrame(halos_of_interest)
    
    

 ----------------------- 100 -----------------------
2025/3/24 at 13:26
gal_100.hdf5 written!
CPU times: user 3min 15s, sys: 1min 15s, total: 4min 30s
Wall time: 1min 22s


# Reading the cloudy gas particle of the most massive halo to compare

In [ ]:
%%time
import importlib
importlib.reload(functions_readfiles)

print(f" -------------------------- For redshift: {redshift} --------------------------")

cloudy_gas_particles = functions_readfiles.read_cloudy_gas_particles(
        galaxy_name = "gal0", 
        galaxy_type = "firebox", 
        redshift = redshift, 
        directory_name = "voronoi_1e6",
        base_fdir = "/scratch/m/murray/dtolgay/post_processing_fire_outputs/skirt/runs_hden_radius"
    )

# Read star particles 
comprehensive_star_particles = functions_readfiles.read_comprehensive_star_particles(
        galaxy_name = "gal0", 
        galaxy_type = "firebox", 
        redshift = redshift, 
        directory_name = "voronoi_1e6",
        base_fdir = "/scratch/m/murray/dtolgay/post_processing_fire_outputs/skirt/runs_hden_radius"
    )

### Comparing filtered_gas_particles with the cloudy_gas_particles

In [ ]:
# sum(filtered_gas_particles['mass']*1e10) / 1e11
sum(filtered_star_particles['mass']*1e10) / 1e11

In [ ]:
# sum(cloudy_gas_particles['mass']) / 1e11
sum(comprehensive_star_particles['mass'] / 1e11)

In [ ]:
#################################### Plot

print("Plotting galaxy")
# Create a figure with two subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))  # Adjust figsize as needed

# First subplot: x-y axis plot
axs[0].hist2d(
    x=filtered_gas_particles["x"], 
    y=filtered_gas_particles["y"], 
    bins=500,
    norm=matplotlib.colors.LogNorm(),
)
axs[0].set_title('Filtered Gas Particles')
axs[0].set_xlabel('x')
axs[0].set_ylabel('y')

# Second subplot: z-y axis plot
axs[1].hist2d(
    x=cloudy_gas_particles["x"],  # Use the 'y' column for the x-axis
    y=cloudy_gas_particles["y"],  # Keep the 'z' column for the y-axis
    bins=500,
    norm=matplotlib.colors.LogNorm(),
#     range=[[-R_max, R_max], [-R_max, R_max]]
)
axs[1].set_title('Cloudy gas particles')
axs[1].set_xlabel('x')
axs[1].set_ylabel('y')

# Show the figure
plt.tight_layout()


In [ ]:
#################################### Plot

print("Plotting galaxy")
# Create a figure with two subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))  # Adjust figsize as needed

# First subplot: x-y axis plot
axs[0].hist2d(
    x=filtered_star_particles["x"], 
    y=filtered_star_particles["y"], 
    bins=500,
    norm=matplotlib.colors.LogNorm(),
)
axs[0].set_title('Filtered Star Particles')
axs[0].set_xlabel('x')
axs[0].set_ylabel('y')

axs[1].hist2d(
    x=comprehensive_star_particles["x"],  # Use the 'y' column for the x-axis
    y=comprehensive_star_particles["y"],  # Keep the 'z' column for the y-axis
    bins=500,
    norm=matplotlib.colors.LogNorm(),
#     range=[[-R_max, R_max], [-R_max, R_max]]
)
axs[1].set_title('Comphrehensive star particles')
axs[1].set_xlabel('x')
axs[1].set_ylabel('y')

# Show the figure
plt.tight_layout()


# Read the file 

In [ ]:
file_name = "gal_0.hdf5"

gas_particles = {}
star_particles = {}

with h5py.File(f'z1.0/{file_name}', 'r') as f:
    for key in f['gas']:
        gas_particles[key] = f['gas'][key][:]
    for key in f['star']:
        star_particles[key] = f['star'][key][:]


In [ ]:
# Create dataframe

gas_particles_read = functions_AHF.create_df_from_read_hdf5(particles = gas_particles, particle_type="gas")
star_particles_read = functions_AHF.create_df_from_read_hdf5(particles = star_particles, particle_type = "star")

In [ ]:
importlib.reload(functions_AHF)

gas_particles_process, star_particles_processed = functions_AHF.process_and_rotate_galaxy(
    gas_particles_df = gas_particles_read.copy(),
    star_particles_df = star_particles_read.copy(),
)

## Plot and check the results

In [ ]:
#################################### Plot

print("Plotting galaxy")
# Create a figure with two subplots
fig, axs = plt.subplots(1, 3, figsize=(25, 6))  # Adjust figsize as needed

# First subplot: x-y axis plot
axs[0].hist2d(
    x=filtered_gas_particles["x"], 
    y=filtered_gas_particles["y"], 
    bins=500,
    norm=matplotlib.colors.LogNorm(),
)
axs[0].set_title('Filtered Gas Particles')
axs[0].set_xlabel('x')
axs[0].set_ylabel('y')

# Second subplot: z-y axis plot
axs[1].hist2d(
    x=cloudy_gas_particles["x"],  # Use the 'y' column for the x-axis
    y=cloudy_gas_particles["y"],  # Keep the 'z' column for the y-axis
    bins=500,
    norm=matplotlib.colors.LogNorm(),
#     range=[[-R_max, R_max], [-R_max, R_max]]
)
axs[1].set_title('Cloudy gas particles')
axs[1].set_xlabel('x')
axs[1].set_ylabel('y')

####
axs[2].hist2d(
    x=gas_particles_process["x"],  # Use the 'y' column for the x-axis
    y=gas_particles_process["y"],  # Keep the 'z' column for the y-axis
    bins=500,
    norm=matplotlib.colors.LogNorm(),
#     range=[[-R_max, R_max], [-R_max, R_max]]
)
axs[2].set_title('Processed galaxy')
axs[2].set_xlabel('x')
axs[2].set_ylabel('y')


# Show the figure
plt.tight_layout()


In [ ]:
print(
    sum(star_particles_processed['mass'])*1e10/1e11, 
    sum(comprehensive_star_particles['mass'])/1e11,
    sum(star_particles_read['mass'])*1e10/1e11,
)